In [1]:
# Library for Data Manipulation
import numpy as np
import pandas as pd

#Library for Data Visualization.
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot

# Library for Statistical Modelling
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error 
from sklearn.linear_model import Ridge,LinearRegression

<div style="text-align: center; background-color: yellow; padding: 10px;">
    <h2 style="font-weight: bold;">LOADING DATASET</h2>
</div>

In [2]:
df=data = pd.read_csv(r'MSB-Mortgage-Backed-Securities-Pipeline-main-LoanExport-Revised1.csv')

In [3]:
df.head()

,MSA,MIP,Units,OCLTV,DTI,OrigUPB,OrigInterestRate,Channel,PPM,PropertyState,...,MaturityYear,MaturityMonth,IsFirstTimeHomebuyer,LTV_Range,CreditRange,RePayRange,LoanPurpose_N,LoanPurpose_P,Occupancy_O,Occupancy_S
0,16974,25,1,89,27,117000,6.750,3,0,15,...,2029,1,0,0,0,3,0,1,1,0
1,19740,0,1,73,17,109000,6.500,2,0,5,...,2029,1,0,0,0,1,1,0,1,0
2,29940,0,1,75,16,88000,6.875,3,0,17,...,2029,1,0,0,0,3,1,0,1,0
3,31084,0,1,76,14,160000,6.875,2,0,4,...,2029,1,0,0,0,0,1,0,1,0
4,35644,0,1,78,18,109000,7.125,2,0,32,...,2029,1,0,0,0,3,1,0,1,0


In [4]:
# Print the shape of the DataFrame
print("The shape of data frame:",df.shape)
# Print the length (number of rows) of the DataFrame
print("Number of Rows in the dataframe:", len(df))
# Print the number of columns in the DataFrame
print("Number of Columns in the dataframe:", len(df))

The shape of data frame: (291403, 30)
Number of Rows in the dataframe: 291403
Number of Columns in the dataframe: 291403


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291403 entries, 0 to 291402
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   MSA                   291403 non-null  int64  
 1   MIP                   291403 non-null  int64  
 2   Units                 291403 non-null  int64  
 3   OCLTV                 291403 non-null  int64  
 4   DTI                   291403 non-null  int64  
 5   OrigUPB               291403 non-null  int64  
 6   OrigInterestRate      291403 non-null  float64
 7   Channel               291403 non-null  int64  
 8   PPM                   291403 non-null  int64  
 9   PropertyState         291403 non-null  int64  
 10  PropertyType          291403 non-null  int64  
 11  OrigLoanTerm          291403 non-null  int64  
 12  NumBorrowers          291403 non-null  int64  
 13  SellerName            291403 non-null  int64  
 14  ServicerName          291403 non-null  int64  
 15  

### <span style='color:blue'>1] MONTHLY INTEREST RATE</span> 

In [6]:
df['OrigInterestRate_Monthly'] =  np.round((df['OrigInterestRate'] / 12) / 100, 4)

### <span style='color:blue'>2] MONTHLY INSTALLMENT</span> 

In [7]:
def calculateEmi(principal, monthly_interest_rate, loan_term_months):
    numerator = (1 + monthly_interest_rate) ** loan_term_months
    denominator = numerator - 1
    interest = numerator / denominator
    emi = principal * monthly_interest_rate * interest
    return np.int64(emi)

In [8]:
df['MonthlyInstallment'] = df.apply(
        lambda features: calculateEmi(
            principal=features['OrigUPB'], 
            monthly_interest_rate=features['OrigInterestRate_Monthly'],
            loan_term_months=features['OrigLoanTerm']), axis=1)

### <span style='color:blue'>3] CURRENT UNPAID PRINCIPAL</span> 

In [9]:
def get_currentUPB(principal, monthly_interest_rate, monthly_installment,
                   payments_made):
    monthly_interest = monthly_interest_rate * principal
    monthly_paid_principal = monthly_installment - monthly_interest
    unpaid_principal = principal - (monthly_paid_principal * payments_made)
    return np.int32(unpaid_principal)

In [10]:
df['CurrentUPB'] = df.apply(
        lambda features: get_currentUPB(
            monthly_interest_rate=features['OrigInterestRate_Monthly'],
            principal=features['OrigUPB'], 
            monthly_installment=features['MonthlyInstallment'],
            payments_made=features['MonthsInRepayment']), axis=1)

### <span style='color:blue'>4] MONTHLY INCOME</span> 

In [11]:
def calculate_monthly_income(dti, emi):
    dti = dti if dti <1 else dti / 100
    # Calculate montly income
    if dti == 0:
        monthly_income = emi
    else:
        monthly_income = emi / dti
    return np.int64 (monthly_income)

In [12]:
df['MonthlyIncome'] = df.apply(
        lambda features: calculate_monthly_income(
            dti = features['DTI'],
            emi= features['MonthlyInstallment']), axis=1)

### <span style='color:blue'>5] PREPAYMENT</span> 

In [13]:
def calculatePrepayment(dti, monthly_income):
    if (dti < 40):
        prepayment = monthly_income / 2;
    else:
        prepayment = monthly_income * 3 / 4;
    return np.int64(prepayment)

In [14]:
df['Prepayment'] = df.apply(
        lambda features: calculatePrepayment(
            dti=features['DTI'],
            monthly_income=features['MonthlyIncome']), axis=1)

In [15]:
df['Prepayment']=(df['Prepayment']*24)-(df['MonthlyInstallment']*24)

### <span style='color:blue'>6] TOTAL PAYMENT AND INTEREST AMOUNT</span> 

In [16]:
df['Totalpayment'] = df['MonthlyInstallment'] * df['OrigLoanTerm']
df['InterestAmount'] = df['Totalpayment'] - df['OrigUPB']

In [17]:
related_columns = ['OrigUPB', 'OrigInterestRate_Monthly', 'OrigLoanTerm',
                   'DTI', 'MonthsInRepayment' , 'MonthlyInstallment',
                   'CurrentUPB', 'MonthlyIncome', 'Prepayment', 'Totalpayment',
                   'InterestAmount']

In [18]:
df[related_columns]

,OrigUPB,OrigInterestRate_Monthly,OrigLoanTerm,DTI,MonthsInRepayment,MonthlyInstallment,CurrentUPB,MonthlyIncome,Prepayment,Totalpayment,InterestAmount
0,117000,0.0056,360,27,52,756,111758,2800,15456,272160,155160
1,109000,0.0054,360,17,144,687,94830,4041,31992,247320,138320
2,88000,0.0057,360,16,67,576,83015,3600,29376,207360,119360
3,160000,0.0057,360,14,35,1047,155275,7478,64608,376920,216920
4,109000,0.0059,360,18,54,731,104253,4061,31176,263160,154160
...,...,...,...,...,...,...,...,...,...,...,...
291398,90000,0.0058,360,32,12,596,89112,1862,8040,214560,124560
291399,267000,0.0053,319,0,53,1736,249992,1736,-20832,553784,286784
291400,87000,0.0047,313,31,38,531,82360,1712,7800,166203,79203
291401,88000,0.0058,360,54,32,583,85676,1079,5424,209880,121880


In [19]:
df.head()

,MSA,MIP,Units,OCLTV,DTI,OrigUPB,OrigInterestRate,Channel,PPM,PropertyState,...,LoanPurpose_P,Occupancy_O,Occupancy_S,OrigInterestRate_Monthly,MonthlyInstallment,CurrentUPB,MonthlyIncome,Prepayment,Totalpayment,InterestAmount
0,16974,25,1,89,27,117000,6.750,3,0,15,...,1,1,0,0.0056,756,111758,2800,15456,272160,155160
1,19740,0,1,73,17,109000,6.500,2,0,5,...,0,1,0,0.0054,687,94830,4041,31992,247320,138320
2,29940,0,1,75,16,88000,6.875,3,0,17,...,0,1,0,0.0057,576,83015,3600,29376,207360,119360
3,31084,0,1,76,14,160000,6.875,2,0,4,...,0,1,0,0.0057,1047,155275,7478,64608,376920,216920
4,35644,0,1,78,18,109000,7.125,2,0,32,...,0,1,0,0.0059,731,104253,4061,31176,263160,154160


<div style="text-align: center; background-color: yellow; padding: 10px;">
    <h2 style="font-weight: bold;">FEATURE SELECTION</h2>
</div>

### <span style='color:blue'>1] ANOVA TEST</span> 

In [20]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split

In [21]:
x= df.drop('Prepayment', axis=1)
y = df.Prepayment

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
scaler = StandardScaler()
x_train_scaled= scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [23]:
selector= SelectKBest(score_func=f_regression,k=10)
x_train_sel=selector.fit_transform(x_train_scaled,y_train)
x_test_sel=selector.transform(x_test_scaled)

In [24]:
p_values=selector.pvalues_
f_values=selector.scores_
results=pd.DataFrame({
    'feature':x.columns,
    'f_values': f_values,
    'p_values':p_values
})

results

,feature,f_values,p_values
0,MSA,4.420541e+01,2.963107e-11
1,MIP,5.203890e+02,4.659324e-115
2,Units,3.400849e+02,6.926374e-76
3,OCLTV,5.873256e+02,1.385347e-129
4,DTI,9.429129e+03,0.000000e+00
5,OrigUPB,1.144600e+04,0.000000e+00
6,OrigInterestRate,1.260519e+01,3.847526e-04
7,Channel,4.855859e+02,1.676434e-107
8,PPM,2.064684e+02,8.480382e-47
9,PropertyState,5.094774e+02,1.089185e-112


In [25]:
top=results.sort_values(by='f_values',ascending=False).head(10).reset_index(drop=True)
top

,feature,f_values,p_values
0,MonthlyIncome,2.416610e+06,0.000000e+00
1,InterestAmount,1.174253e+04,0.000000e+00
2,Totalpayment,1.168331e+04,0.000000e+00
3,MonthlyInstallment,1.168039e+04,0.000000e+00
4,OrigUPB,1.144600e+04,0.000000e+00
5,CurrentUPB,1.108085e+04,0.000000e+00
6,DTI,9.429129e+03,0.000000e+00
7,NumBorrowers,2.398520e+03,0.000000e+00
8,LoanPurpose_N,7.040140e+02,6.821344e-155
9,OCLTV,5.873256e+02,1.385347e-129


In [26]:
def evaluate(model, x_train, x_test, y_train, y_test):
    y_test_pred = model.predict(x_test)
    y_train_pred = model.predict(x_train)

    print("TRAINIG RESULTS: \n===============================")
    print(f"Mean Squared Error:\n{mean_squared_error(y_train, y_train_pred)}")
    print(f"Mean Absolute Error:\n{mean_absolute_error(y_train, y_train_pred)}")
    print(f"R2 Score:\n{r2_score(y_train, y_train_pred)}")

    print("TESTING RESULTS: \n===============================")
    print(f"Mean Squared Error:\n{mean_squared_error(y_test, y_test_pred)}")
    print(f"Mean Absolute Error:\n{mean_absolute_error(y_test, y_test_pred)}")
    print(f"R2 Score:\n{r2_score(y_test, y_test_pred)}")

<div style="text-align: center; background-color: yellow; padding: 10px;">
    <h2 style="font-weight: bold;">LINEAR REGRESSION</h2>
</div>

In [27]:
model_lin=LinearRegression()
# model creation
model_lin=model_lin.fit(x_train_sel,y_train)
y_pred_lin=model_lin.predict(x_test_sel)

In [28]:
evaluate(model_lin, x_train_sel, x_test_sel, y_train, y_test)

TRAINIG RESULTS: 
Mean Squared Error:
16596586.258248014
Mean Absolute Error:
3155.408731807573
R2 Score:
0.9743266840938732
TESTING RESULTS: 
Mean Squared Error:
16804501.773315445
Mean Absolute Error:
3165.998492389942
R2 Score:
0.9718300796827734


<div style="text-align: center; background-color: yellow; padding: 10px;">
    <h2 style="font-weight: bold;">RIDGE REGRESSION</h2>
</div>

In [29]:
model_ridge=Ridge()
model_ridge.fit(x_train_sel,y_train)
y_pred_ridge=model_ridge.predict(x_test_sel)

In [30]:
evaluate(model_ridge, x_train_sel, x_test_sel, y_train, y_test)

TRAINIG RESULTS: 
Mean Squared Error:
16596588.999335254
Mean Absolute Error:
3155.397736352586
R2 Score:
0.974326679853676
TESTING RESULTS: 
Mean Squared Error:
16804496.433531284
Mean Absolute Error:
3165.9874515471247
R2 Score:
0.9718300886340232
